# Ventas de las tiendas vs distancia

### Código todo en uno ejecutable

In [1]:
import pandas as pd
import math

# Importamos las ventas por CP de nuestras tiendas (incluye nombre de la tienda, CP de la tienda, CP de la venta, cantidad vendida):
sales = pd.read_csv('Sales_per_point.csv',sep=",",decimal=".")

# Importamos las coordenadas de todos los CP de España:
coord = pd.read_csv('02_Output_Codigos_Postales.csv',sep=",",decimal=".", dtype={'CP':'int', 'Market Size':'float','Latitude':'float','Longitude':'float'})

# Incluimos para cada CP de la tienda y CP de venta sus coordenadas:
sales2 = sales.merge(coord, left_on='CP CMP', right_on='CP', how='left')
sales2.rename(columns={'Latitude':'lata', 'Longitude':'lona'},inplace=True)
sales2.drop(['CP','Market Size'], inplace=True, axis=1)

df = sales2.merge(coord, left_on='CP Sales', right_on='CP', how='left')
df.rename(columns={'Latitude':'latb', 'Longitude':'lonb'},inplace=True)
df.drop('CP', inplace=True, axis=1)

# Calculamos la distancia entre el CP de la tienda y de la venta basándonos en las coordenadas:
df2 = pd.DataFrame(columns=['CMP', 'CP1', 'CP2', 'Dist', 'Roundist', 'Sales', 'Market Size'])
line=1

for cp,row in df.iterrows():
    cmp = row[1]
    cp1 = row[2]
    cp2 = row[3]
    sales = row[4]
    ms = row[7]
    lata = row[5]
    lona = row[6]
    latb = row[8]
    lonb = row[9]
    try:
        dist = math.acos(math.cos(math.radians(90-lata)) * math.cos(math.radians(90-latb)) + math.sin(math.radians(90-lata)) * math.sin(math.radians(90-latb)) * math.cos(math.radians(lona-lonb))) * 6371
    except:
        dist = 0
    rdist = round(dist)
    df2.loc[line] = (cmp, cp1, cp2, dist, rdist, sales, ms)
    line +=1 
    
# Agrupamos las ventas por distancia
Dist_sales = df2.groupby(['CMP','Roundist'],as_index=False).agg({'Sales':'sum', 'Market Size':'sum'}) 

#Añadimos las ventas acumuladas y el % que supone del total. Eliminamos aquellos CP demasiado alejados
Dist_sales['Pct'] = (Dist_sales['Sales']/Dist_sales['Sales'].sum())
Dist_sales['Acc_sales'] = Dist_sales['Sales'].cumsum()
Dist_sales['Acc_Pct'] = (Dist_sales['Acc_sales']/Dist_sales['Sales'].sum())
Dist_sales['Acc_MSize'] = Dist_sales['Market Size'].cumsum()
Dist_sales['MShare'] = Dist_sales['Sales']/Dist_sales['Market Size']
Dist_sales['Acc_MShare'] = Dist_sales['Acc_sales']/Dist_sales['Acc_MSize']

CMP_Dist_Sales = Dist_sales[Dist_sales.Roundist < 21]

# Exportamos el resultado a un CSV
CMP_Dist_Sales.to_csv('04_Output_CMP_Dist_sales.csv', sep=',',decimal=".", index=False)

# Exportamos las ventas y Market Share actuales a un CSV
CP_MShare = df2.groupby(['CP2'],as_index=False).agg({'Sales':'sum', 'Market Size':'mean'})
CP_MShare['Actual_MarketShare'] = CP_MShare['Sales']/CP_MShare['Market Size']
CP_MShare.drop(['Sales','Market Size'], inplace=True, axis=1)
CP_MShare.to_csv('04_Output_CP_Actual_MarketShare.csv', sep=',',decimal=".", index=False)

### Ejercicio por partes

Juntamos las ventas por CP por cada Punto de venta y las coordenadas, para saber la venta por distancia y el tamaño del mercado

In [2]:
import pandas as pd
import math

# Importamos las ventas por CP de nuestras tiendas (incluye nombre de la tienda, CP de la tienda, CP de la venta, cantidad vendida):
sales = pd.read_csv('Sales_per_point.csv',sep=",",decimal=".")

# Importamos las coordenadas de todos los CP de España:
coord = pd.read_csv('02_Output_Codigos_Postales.csv',sep=",",decimal=".", dtype={'CP':'int', 'Market Size':'float','Latitude':'float','Longitude':'float'})

# Incluimos para cada CP de la tienda y CP de venta sus coordenadas:
sales2 = sales.merge(coord, left_on='CP CMP', right_on='CP', how='left')
sales2.rename(columns={'Latitude':'lata', 'Longitude':'lona'},inplace=True)
sales2.drop(['CP','Market Size'], inplace=True, axis=1)

df = sales2.merge(coord, left_on='CP Sales', right_on='CP', how='left')
df.rename(columns={'Latitude':'latb', 'Longitude':'lonb'},inplace=True)
df.drop('CP', inplace=True, axis=1)

# Calculamos la distancia entre el CP de la tienda y de la venta basándonos en las coordenadas:
df2 = pd.DataFrame(columns=['CMP', 'CP1', 'CP2', 'Dist', 'Roundist', 'Sales', 'Market Size'])
line=1

for cp,row in df.iterrows():
    cmp = row[1]
    cp1 = row[2]
    cp2 = row[3]
    sales = row[4]
    ms = row[7]
    lata = row[5]
    lona = row[6]
    latb = row[8]
    lonb = row[9]
    try:
        dist = math.acos(math.cos(math.radians(90-lata)) * math.cos(math.radians(90-latb)) + math.sin(math.radians(90-lata)) * math.sin(math.radians(90-latb)) * math.cos(math.radians(lona-lonb))) * 6371
    except:
        dist = 0
    rdist = round(dist)
    df2.loc[line] = (cmp, cp1, cp2, dist, rdist, sales, ms)
    line +=1 
df2.head()

,CMP,CP1,CP2,Dist,Roundist,Sales,Market Size
1,Centro 1,3015,2001,137.564482,138,432.1,4700638.0
2,Centro 1,3015,2005,138.722377,139,134.4,11214889.0
3,Centro 1,3015,2006,149.034247,149,1035.6,8159006.0
4,Centro 1,3015,2008,138.719434,139,50.8,4868566.0
5,Centro 1,3015,2480,160.273566,160,3127.4,293611.0


Comprobaciones. Distancia a la que vendemos >90%, y el MSize caiga por debajo del 5%

In [3]:
# Agrupamos las ventas por distancia
Dist_sales = df2.groupby(['Roundist'],as_index=False).agg({'Sales':'sum', 'Market Size':'sum'}) 

#Añadimos las ventas acumuladas y el % que supone del total
Dist_sales['Pct'] = (Dist_sales['Sales']/Dist_sales['Sales'].sum())
Dist_sales['Acc_sales'] = Dist_sales['Sales'].cumsum()
Dist_sales['Acc_Pct'] = (Dist_sales['Acc_sales']/Dist_sales['Sales'].sum())
Dist_sales['Acc_MSize'] = Dist_sales['Market Size'].cumsum()
Dist_sales['MShare'] = Dist_sales['Sales']/Dist_sales['Market Size']
Dist_sales['Acc_MShare'] = Dist_sales['Acc_sales']/Dist_sales['Acc_MSize']

Dist_sales.head(40)

,Roundist,Sales,Market Size,Pct,Acc_sales,Acc_Pct,Acc_MSize,MShare,Acc_MShare
0,0,17135404.26,158086458.0,0.106755,1.713540e+07,0.106755,1.580865e+08,0.108393,0.108393
1,1,9696824.62,112261733.0,0.060412,2.683223e+07,0.167167,2.703482e+08,0.086377,0.099251
2,2,18338614.85,210837417.0,0.114251,4.517084e+07,0.281418,4.811856e+08,0.086980,0.093874
3,3,22140172.67,265388777.0,0.137935,6.731102e+07,0.419353,7.465744e+08,0.083425,0.090160
4,4,15799690.02,251894613.0,0.098433,8.311071e+07,0.517786,9.984690e+08,0.062723,0.083238
5,5,15070711.80,216068992.0,0.093892,9.818142e+07,0.611677,1.214538e+09,0.069750,0.080838
6,6,8186110.43,176863067.0,0.051000,1.063675e+08,0.662677,1.391401e+09,0.046285,0.076446
7,7,8241352.91,156571350.0,0.051344,1.146089e+08,0.714022,1.547972e+09,0.052636,0.074038
8,8,3409156.38,67885478.0,0.021239,1.180180e+08,0.735261,1.615858e+09,0.050219,0.073037
9,9,6021198.85,126400925.0,0.037513,1.240392e+08,0.772774,1.742259e+09,0.047636,0.071194


Exportamos para el siguiente paso

In [4]:
# Agrupamos las ventas por distancia
Dist_sales = df2.groupby(['CMP','Roundist'],as_index=False).agg({'Sales':'sum', 'Market Size':'sum'}) 

#Añadimos las ventas acumuladas y el % que supone del total
Dist_sales['Pct'] = (Dist_sales['Sales']/Dist_sales['Sales'].sum())
Dist_sales['Acc_sales'] = Dist_sales['Sales'].cumsum()
Dist_sales['Acc_Pct'] = (Dist_sales['Acc_sales']/Dist_sales['Sales'].sum())
Dist_sales['Acc_MSize'] = Dist_sales['Market Size'].cumsum()
Dist_sales['MShare'] = Dist_sales['Sales']/Dist_sales['Market Size']
Dist_sales['Acc_MShare'] = Dist_sales['Acc_sales']/Dist_sales['Acc_MSize']

CMP_Dist_Sales = Dist_sales[Dist_sales.Roundist < 21]
CMP_Dist_Sales.head(30)

,CMP,Roundist,Sales,Market Size,Pct,Acc_sales,Acc_Pct,Acc_MSize,MShare,Acc_MShare
0,Centro 1,0,846265.32,6516160.0,0.005272,846265.32,0.005272,6.516160e+06,0.129872,0.129872
1,Centro 1,1,202173.37,2754035.0,0.001260,1048438.69,0.006532,9.270195e+06,0.073410,0.113098
2,Centro 1,2,2118257.94,26909635.0,0.013197,3166696.63,0.019729,3.617983e+07,0.078717,0.087527
3,Centro 1,3,1218806.81,13314628.0,0.007593,4385503.44,0.027322,4.949446e+07,0.091539,0.088606
4,Centro 1,4,1503756.57,15605252.0,0.009369,5889260.01,0.036691,6.509971e+07,0.096362,0.090465
5,Centro 1,5,2102056.15,28157079.0,0.013096,7991316.16,0.049786,9.325679e+07,0.074655,0.085692
6,Centro 1,6,464069.79,20668245.0,0.002891,8455385.95,0.052678,1.139250e+08,0.022453,0.074219
7,Centro 1,7,441519.60,6914736.0,0.002751,8896905.55,0.055428,1.208398e+08,0.063852,0.073626
8,Centro 1,9,797226.53,15933681.0,0.004967,9694132.08,0.060395,1.367735e+08,0.050034,0.070877
9,Centro 1,10,81068.15,1378146.0,0.000505,9775200.23,0.060900,1.381516e+08,0.058824,0.070757


In [5]:
# Exportamos el resultado a un CSV
CMP_Dist_Sales.to_csv('04_Output_CMP_Dist_sales.csv', sep=',',decimal=".", index=False)

Exportamos para el último paso

In [6]:
CP_MShare = df2.groupby(['CP2'],as_index=False).agg({'Sales':'sum', 'Market Size':'mean'})
CP_MShare['Actual_MarketShare'] = CP_MShare['Sales']/CP_MShare['Market Size']
CP_MShare.drop(['Sales','Market Size'], inplace=True, axis=1)
CP_MShare.to_csv('04_Output_CP_Actual_MarketShare.csv', sep=',',decimal=".", index=False)